##  graph with writer-critic loop

### Subtask:
Modify the `build_joke_graph` function to:
- Add the `writer_node` and `critic_node`.
- Route the `fetch_joke` choice to the `writer_node`.
- Add a conditional edge from the `critic_node` to either `show_final_joke` (if approved) or back to `writer_node` (if rejected and retries are less than the limit).
- Add an edge from `writer_node` to `critic_node`.
- Add a node for `show_final_joke` to display the approved joke.
- Add logic to reset the evaluation state after `show_final_joke` or when the category/language changes.
- Add a maximum retry limit (e.g., 5).

## define a state

In [71]:
from pydantic import BaseModel
from typing import List, Literal, Annotated

add = "placeholder_for_add_logic"

class Joke(BaseModel):
  text:str
  category: str


class JokeState(BaseModel):
    jokes: Annotated[List[Joke], add] = []
    jokes_choice: Literal["n", "c", "q", "l"] = "n" # next joke, change category, change language or quit
    category: str = "neutral"
    language: str = "en"
    quit: bool = False
    latest_joke: str = ""
    approval_status: Literal["approved", "rejected", "needs_review"] = "needs_review" # Add approval_status field
    retry_count: int = 0 # Add retry_count field
    joke_history: List[str] = [] # Add joke_history field


## Add node functions

In [76]:
from groq import Groq
from google.colab import userdata
client = Groq(api_key=userdata.get('GROQ_API_KEY'))


def show_menu(state: JokeState) -> dict:
    user_input = input("[n] Next  [c] Category [l] Language [q] Quit\n> ").strip().lower()
    return {"jokes_choice": user_input}

def fetch_joke(state: JokeState) -> dict:
    joke_text = get_joke(language=state.language, category=state.category)
    new_joke = Joke(text=joke_text, category=state.category)
    return {"jokes": [new_joke]}
def update_category(state: JokeState) -> dict:
    categories = ["neutral", "chuck", "all"]
    selection = int(input("Select category [0=neutral, 1=chuck, 2=all]: ").strip())
    return {"category": categories[selection]}


def exit_bot(state: JokeState) -> dict:
    return {"quit": True}

def change_language(state: JokeState) -> dict:
    available_languages = ["en", "es", "fr", "de","tr"]
    print("Select language:")
    for i, lang in enumerate(available_languages):
        print(f"[{i}] {lang}")
    selection = int(input("> ").strip())
    return {"language": available_languages[selection]}

def writer_node(state: JokeState) -> dict:
    """
    Generates a joke using an LLM based on the current category and updates the state.
    """
    print("Generating a joke...")
    max_regeneration_attempts = 5
    for attempt in range(max_regeneration_attempts):

      try:
          # Use the prompt builder utility here when implemented
          prompt = f"Tell me a {state.category} joke in {state.language}."
          chat_completion = client.chat.completions.create(
              messages=[
                  {
                      "role": "user",
                      "content": prompt,
                  }
              ],
              model="llama3-8b-8192", # Using a Groq model
          )
          joke_text = chat_completion.choices[0].message.content
          if joke_text not in state.joke_history:
            print(f"Generated unique joke on attempt {attempt + 1}.")
            return {"latest_joke": joke_text, "approval_status": "needs_review", "retry_count": 0}
          else:
            print(f"Joke repeated. Attempting to regenerate... (Attempt {attempt + 1})")

          # Update the state with the generated joke and set initial approval status
          return {"latest_joke": joke_text, "approval_status": "needs_review", "retry_count": 0}

      except Exception as e:
          print(f"Error generating joke: {e}")
          return {"latest_joke": "Could not generate a joke.", "approval_status": "rejected", "retry_count": state.retry_count + 1}
    print(f"Failed to generate a unique joke after {max_regeneration_attempts} attempts.")
    return {"latest_joke": "Could not generate a unique joke.", "approval_status": "rejected", "retry_count": state.retry_count + 1}


def critic_node(state: JokeState) -> dict:
    """
    Evaluates the latest joke using an LLM and updates the approval status in the state.
    """
    print("Critiquing the joke...")
    try:
        # Use the prompt builder utility here when implemented
        prompt = build_prompt("critic", state) # Use the prompt builder

        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="llama3-8b-8192", # Using a Groq model
        )
        critique = chat_completion.choices[0].message.content.strip().lower()

        # Determine approval status based on the critique
        if "approved" in critique:
            return {"approval_status": "approved"}
        else:
            return {"approval_status": "rejected", "retry_count": state.retry_count + 1} # Increment retry count on rejection

    except Exception as e:
        print(f"Error critiquing joke: {e}")
        # Handle errors during critique, potentially rejecting the joke
        return {"approval_status": "rejected", "retry_count": state.retry_count + 1}

def show_final_joke(state: JokeState) -> dict:
    """
    Displays the approved joke and resets evaluation state.
    """
    print(state.latest_joke)
    # Reset evaluation state
    updated_history = state.joke_history + [state.latest_joke]
    # Reset evaluation state
    return {"latest_joke": "", "approval_status": "needs_review", "retry_count": 0, "joke_history": updated_history}




def route_critic_output(state: JokeState) -> str:
    """
    Routes based on the critic's approval status and retry count.
    """
    if state.approval_status == "approved":
        return "show_final_joke"
    elif state.retry_count < 5:  # Set a retry limit
        print(f"Joke rejected. Retrying... (Attempt {state.retry_count})")
        return "writer_node"
    else:
        print("Failed to generate an approved joke after multiple retries.")
        return "show_menu" # Return to menu after too many retries



## Add route

In [73]:
def route_choice(state: JokeState) -> str:
    if state.jokes_choice == "n":
        return "fetch_joke"
    elif state.jokes_choice == "c":
        return "update_category"
    elif state.jokes_choice == "l":
        return "change_language"
    elif state.jokes_choice == "q":
        return "exit_bot"
    return "exit_bot"



## Create a graph and add nodes+edges


In [74]:
from langgraph.graph import StateGraph, END
from langgraph.graph.state import CompiledStateGraph


def build_joke_graph() -> CompiledStateGraph:
    workflow = StateGraph(JokeState)

    workflow.add_node("show_menu", show_menu)
    workflow.add_node("writer_node", writer_node) # Added writer node
    workflow.add_node("critic_node", critic_node) # Added critic node
    workflow.add_node("fetch_joke", fetch_joke)
    workflow.add_node("update_category", update_category)
    workflow.add_node("exit_bot", exit_bot)
    workflow.add_node("change_language", change_language)
    workflow.add_node("show_final_joke", show_final_joke) # Added show_final_joke node


    workflow.set_entry_point("show_menu")

    workflow.add_conditional_edges(
        "show_menu",
        route_choice,
        {
            "fetch_joke": "writer_node",
            "update_category": "update_category",
            "exit_bot": "exit_bot",
            "change_language": "change_language", # Added the conditional edge for change_language
        }
    )
    workflow.add_conditional_edges(
        "critic_node",
        route_critic_output,
        {
            "show_final_joke": "show_final_joke",  # Approved jokes go to show_final_joke
            "writer_node": "writer_node",          # Rejected jokes go back to writer (if retries < limit)
            "show_menu": "show_menu",              # Go to menu if retries >= limit
        }
    )

    workflow.add_edge("fetch_joke", "show_menu")
    workflow.add_edge("update_category", "show_menu")
    workflow.add_edge("change_language", "show_menu") # Added edge from change_language back to show_menu
    workflow.add_edge("writer_node", "critic_node") # Writer goes to Critic
    workflow.add_edge("show_final_joke", "show_menu") # After showing joke, go back to menu
    workflow.add_edge("exit_bot", END)

    return workflow.compile()

In [ ]:
def main():
    graph = build_joke_graph()
    print("Starting the joke bot...")
    final_state = graph.invoke(JokeState(), config={"recursion_limit": 100})
    print("\nJoke bot finished.")
    print("Final state:", final_state)

# Call the main function to run the bot
if __name__ == "__main__":
    main()

Starting the joke bot...
[n] Next  [c] Category [l] Language [q] Quit
> n
Generating a joke...
Generated unique joke on attempt 1.
Critiquing the joke...
Joke being critiqued: Why did the chicken cross the playground?

To get to the other slide!
Critic output: **approved**

i approve of this joke because it is a clever play on words. the traditional punchline to the classic joke "why did the chicken cross the road?" is "to get to the other side!", but this joke takes that and gives it a creative twist by replacing "road" with "playground" and "side" with "slide". the result is a joke that is both familiar and fresh, making it enjoyable to hear or read. the joke requires a basic understanding of the original joke and the wordplay between "side" and "slide", which adds to its cleverness and makes it more likely to elicit a smile or a chuckle.
Why did the chicken cross the playground?

To get to the other slide!
[n] Next  [c] Category [l] Language [q] Quit
> c
Select category [0=neutral, 